## Jihočeský kraj

In [14]:
import pandas
import xml.etree.ElementTree as ET
import re
xml_soubor = 'jihocesky_puvodni.xml'

In [15]:

tree = ET.parse('jihocesky_puvodni.xml')
root = tree.getroot()

skoly = []
for entry in root.findall('PravniSubjekt'):
    skola_group = entry.find('SkolyZarizeni')
    for skola in skola_group.findall('SkolaZarizeni'):
        izo = skola.find('IZO').text
        typ = skola.find('SkolaDruhTyp').text
        nazev = skola.find('SkolaPlnyNazev').text
        adresa = skola.find('SkolaMistaVykonuCinnosti')
        actual_ad = adresa.find('SkolaMistoVykonuCinnosti')
        adresa1 =  actual_ad.find('MistoAdresa1').text
        adresa2 =  actual_ad.find('MistoAdresa2').text
        adresa3 =  actual_ad.find('MistoAdresa3').text
        if adresa3 == None:
            adresa3 = adresa2
        #izoluji název obce regulárním výrazem a ukládám do sloupce obec
        pattern1 = r' \d\d '
        list_obec = re.split(pattern1, adresa3)
        obec = list_obec[-1]
        #izoluji psč
        pattern2 = r' [A-ZÁČĎŘŠŤÚŽ]'
        list_obec = re.split(pattern2, adresa3)
        PSC = list_obec[0]
        #přidávám sloupec kraj
        kraj = 'Jihočeský kraj'
        
        if typ == 'C00':  # číselník pro SŠ
            obor = skola.find('SkolaOboryVzdelani')
            actual_obor = obor.find('SkolaOborVzdelani')
            obor_kod = actual_obor.find('OborKod').text
            obor_nazev = actual_obor.find('OborNazev').text
            obor_forma = actual_obor.find('FormaVzdelavani').text
            obor_delka = actual_obor.find('DelkaVzdelavani').text
            obor_kapacita = actual_obor.find('OborKapacita').text
            
            skoly.append([izo, nazev, typ, adresa1, adresa2, adresa3, PSC, obec, kraj, obor_kod, obor_nazev, obor_forma, obor_delka, obor_kapacita])

sloupce = ['id', 'nazev', 'typ', 'adresa1', 'adresa2', 'adresa3', 'PSC', 'obec', 'kraj', 'obor_kod', 'obor_nazev', 'obor_forma', 'obor_delka', 'obor_kapacita']
df_skoly_stredocesky = pandas.DataFrame(skoly, columns = sloupce)

pandas.set_option('display.max_rows', None)  # chci vidět všechny řádky
df_skoly_jihocesky

NameError: name 'df_skoly_jihocesky' is not defined

In [5]:
df_skoly_jihocesky['adresa3'].isna().sum()


0

In [6]:
df_skoly_jihocesky['adresa3'].unique()

array(['256 01 Benešov', '258 01 Vlašim', '257 56 Neveklov',
       '273 08 Pchery', '266 01 Beroun', '268 01 Hořovice',
       '272 01 Kladno', '274 01 Slaný', '273 03 Stochov', '280 02 Kolín',
       '282 01 Český Brod', '286 01 Čáslav', '284 01 Kutná Hora',
       '277 11 Neratovice', '276 01 Mělník', '278 01 Kralupy nad Vltavou',
       '277 21 Liběchov', '293 01 Mladá Boleslav',
       '295 01 Mnichovo Hradiště', '293 06 Kosmonosy',
       '294 73 Horky nad Jizerou', '294 11 Loukovec', '290 01 Poděbrady',
       '288 02 Nymburk', '289 03 Městec Králové', '289 22 Lysá nad Labem',
       '250 88 Čelákovice', '250 70 Odolena Voda',
       '250 01 Brandýs nad Labem-Stará Boleslav', '251 01 Říčany',
       '254 01 Jílové u Prahy', '263 01 Dobříš', '261 01 Příbram',
       '264 80 Sedlčany', '262 72 Březnice', '262 22 Hluboš',
       '261 01 Dubno', '269 01 Rakovník', '271 01 Nové Strašecí',
       '270 23 Křivoklát', '272 03 Kladno',
       '281 63 Kostelec nad Černými lesy', '251 64 K

přidávám sloupec okres a PSC

In [7]:
vazba_obec_psc_okres = pandas.read_csv("vazba_obec_psc_okres.csv")



In [8]:
df_jihocesky_kraj_final = pandas.merge(df_skoly_jihocesky, vazba_obec_psc_okres, on=['PSC', 'obec'], how='left')


In [9]:
df_jihocesky_kraj_final['okres'].isna().sum()

3

In [10]:
pandas.set_option('display.max_rows', None)
df_jihocesky_kraj_final

,id,nazev,typ,adresa1,adresa2,adresa3,PSC,obec,kraj,obor_kod,obor_nazev,obor_forma,obor_delka,obor_kapacita,okres
0,061664707,Střední škola,C00,Husova 470,256 01 Benešov,256 01 Benešov,256 01,Benešov,Středočeský kraj,7941K41,Gymnázium,10,40,300,Benešov
1,061664651,Střední škola,C00,Mendelova 131,256 01 Benešov,256 01 Benešov,256 01,Benešov,Středočeský kraj,4141M01,Agropodnikání,10,40,250,Benešov
2,061664553,Střední škola,C00,Komenského 41,258 01 Vlašim,258 01 Vlašim,258 01,Vlašim,Středočeský kraj,2341M01,Strojírenství,10,40,240,Benešov
3,061664537,Střední škola,C00,V Sadě 1565,258 01 Vlašim,258 01 Vlašim,258 01,Vlašim,Středočeský kraj,6341M02,Obchodní akademie,10,40,363,Benešov
4,061664715,Střední škola,C00,Černoleská 1997,256 01 Benešov,256 01 Benešov,256 01,Benešov,Středočeský kraj,5341M01,Zdravotnický asistent,10,40,160,Benešov
5,107820013,Střední škola,C00,Jana Nohy 1302,256 01 Benešov,256 01 Benešov,256 01,Benešov,Středočeský kraj,3159E01,Šití oděvů,10,30,18,Benešov
6,061664545,Střední škola,C00,Tylova 271,258 01 Vlašim,258 01 Vlašim,258 01,Vlašim,Středočeský kraj,7941K41,Gymnázium,10,40,272,Benešov
7,107820340,Střední škola,C00,Zámek 1,258 01 Vlašim,258 01 Vlašim,258 01,Vlašim,Středočeský kraj,2951E01,Potravinářská výroba,10,30,48,Benešov
8,110450311,Střední škola,C00,Školní 303,257 56 Neveklov,257 56 Neveklov,257 56,Neveklov,Středočeský kraj,6341M01,Ekonomika a podnikání,10,40,80,Benešov
9,061662356,Střední škola,C00,Humny 338,273 08 Pchery,273 08 Pchery,273 08,Pchery,Středočeský kraj,6542M02,Cestovní ruch,10,40,100,Kladno


In [11]:
df_jihocesky_kraj_final.loc[78, 'okres'] = 'Příbram'


In [12]:
df_jihocesky_kraj_final['okres'].isna().sum()

0

In [13]:
#ukládám do csv
df_jihocesky_kraj_final.to_csv('skoly_Jihocesky_final.csv', index = None)